- 生成数据集

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4]) # 存在两个变量x1、x2, true_w 分别是它们的系数
true_b = 4.2
# 基于真实值 true_w, true_b 生成的样本, 1000 num_examples 样本的大小
features, labels = d2l.synthetic_data(true_w, true_b, 1000) # 如果 y=Aw+b, 则A是features 1000 * 2的数据集, y是labels 1000 * 1的数据集

- 读取数据集

In [3]:
#@save
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器
    从数据集 data_arrays 中按照 batch_size 批次大小获取数据

    将输入的张量数据（如特征和标签）包装成 PyTorch 数据集, 这里是包装 data_arrays
    """
    dataset = data.TensorDataset(*data_arrays)
    # shuffle参数 控制数据集的获取是否随机获取
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

# 测试取第一个数据集
# print(next(iter(data_iter)))

[tensor([[-0.3052, -0.9843],
        [-0.6554, -0.2606],
        [-1.1948,  0.6424],
        [ 0.1302, -0.8806],
        [ 0.0232,  0.8954],
        [ 1.5622,  0.7179],
        [ 0.5960,  0.6039],
        [ 1.0033, -0.3787],
        [ 1.3010, -0.3387],
        [-1.9440,  0.0229]]), tensor([[ 6.9232],
        [ 3.7697],
        [-0.3732],
        [ 7.4371],
        [ 1.2133],
        [ 4.8890],
        [ 3.3437],
        [ 7.4852],
        [ 7.9377],
        [ 0.2300]])]


- 定义模型

In [8]:
from torch import nn

# 线性回归模型, 输入两个变量, 输出一个变量
# y = Aw +b, A表示输入的两个变量, y表示输出的变量
net = nn.Sequential(nn.Linear(2, 1))

- 初始化模型参数

In [9]:
# 在新版本的PyTorch中，net[0].weight.data返回的是一个普通的tensor，而不是可以原地修改的tensor对象，因此没有normal()方法

nn.init.normal_(net[0].weight, mean=0, std=0.01) # net[0] 的权重符合正态分布, 均值为0, 标准差为0.01
nn.init.zeros_(net[0].bias)

Parameter containing:
tensor([0.], requires_grad=True)

- 定义损失函数

In [10]:
loss = nn.MSELoss(reduction='mean') # 使用均方误差损失函数

- 定义优化算法

In [11]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

- 训练

In [12]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter: # 迭代数据集 来更新模型参数 w, b
        l = loss(net(X), y) # 预估值net(x) 和 实际值y
        trainer.zero_grad() # 梯度清零
        l.backward() # 反向传播计算梯度
        trainer.step() # 更新模型参数
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000185
epoch 2, loss 0.000102
epoch 3, loss 0.000102


In [17]:
w = net[0].weight.data
b = net[0].bias.data
print(true_w, true_b)
print(w, b)

print('w的估计误差:', true_w - w.reshape(true_w.shape))
print('b的估计误差:', true_b - b)

tensor([ 2.0000, -3.4000]) 4.2
tensor([[ 1.9995, -3.3990]]) tensor([4.2000])
w的估计误差: tensor([ 0.0005, -0.0010])
b的估计误差: tensor([3.1471e-05])
